In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,19888.45,-0.14,-0.02,0.10,-0.00,-0.02,0.00,-0.00,-0.00
3,-0.14,4668.62,-0.03,0.01,-0.02,0.00,0.01,0.00,0.00
4,-0.02,-0.03,1090.00,0.02,-0.00,-0.00,-0.00,-0.00,0.00
5,0.10,0.01,0.02,255.24,0.00,-0.00,0.00,0.00,0.00
6,-0.00,-0.02,-0.00,0.00,57.40,-0.00,-0.00,0.00,0.00
7,-0.02,0.00,-0.00,-0.00,-0.00,12.64,0.00,0.00,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.80,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.64,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,-0.00000,-0.00003,0.00002,-0.00002,-0.00001
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00007,0.00000,0.00002
4,-0.00000,-0.00001,1.00000,0.00004,-0.00000,-0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00001,-0.00003,0.00007,0.00001,0.00002
6,-0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00001,-0.00000,-0.00003,-0.00001,1.00000,0.00002,0.00001,0.00001
8,0.00002,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00004,0.00001
9,-0.00002,0.00000,-0.00001,0.00001,0.00003,0.00001,0.00004,1.00000,-0.00000
10,-0.00001,0.00002,0.00002,0.00002,0.00001,0.00001,0.00001,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.24724712061707485

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.39922822e+07 2.38475925e+06 1.27356078e+05 6.79029683e+03
 3.29818589e+02 1.50216108e+01 6.61270924e-01 2.70289962e-02
 6.87614909e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999495,-0.031003,-0.006812,-0.001585,-0.000357,-0.000080,-0.000017,-0.000004,-9.570814e-07
1,0.030729,0.998867,-0.035439,-0.007804,-0.001753,-0.000385,-0.000084,-0.000020,-4.491484e-06
2,0.007817,0.034844,0.998461,-0.041468,-0.008763,-0.001924,-0.000430,-0.000098,-2.261568e-05
3,0.002124,0.009087,0.040675,0.997942,-0.047612,-0.009886,-0.002178,-0.000502,-1.165787e-04
4,0.000573,0.002444,0.010476,0.046517,0.997090,-0.058108,-0.012149,-0.002780,-6.475252e-04
5,0.000158,0.000668,0.002861,0.012247,0.056393,0.995139,-0.077868,-0.016779,-3.893767e-03
6,0.000046,0.000193,0.000836,0.003554,0.015863,0.074355,0.989499,-0.120114,-2.588747e-02
7,0.000015,0.000064,0.000274,0.001168,0.005197,0.023641,0.109402,0.966436,-2.311840e-01
8,0.000007,0.000031,0.000134,0.000571,0.002541,0.011543,0.052024,0.226462,9.725576e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005052545533282826,
 0.0011328144017688224,
 0.001538942322381831,
 0.002057699117364198,
 0.0029103254503342058,
 0.004861311642394117,
 0.010500804135188169,
 0.033564412120327236,
 0.027442435639938756]